accessing trained model

In [ ]:
!aws s3 cp s3://fireguarddata/models/tabnet/pytorch-training-2025-03-30-06-45-49-350/output/model.tar.gz ./

download: s3://fireguarddata/models/tabnet/pytorch-training-2025-03-30-06-45-49-350/output/model.tar.gz to ./model.tar.gz
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.creationtime'
tabnet_model.pkl


In [7]:
!tar -tzf model.tar.gz

tar: Ignoring unknown extended header keyword 'LIBARCHIVE.creationtime'
tabnet_model.pkl


## Deploying the model

In [ ]:
# Deploying model with inference script
import sagemaker
from sagemaker.pytorch import PyTorchModel
import boto3

# Initialize SageMaker session
sagemaker_session = sagemaker.Session()
role = "AmazonSageMaker-ExecutionRole-""

# S3 path to your trained model
model_path = "s3://fireguarddata/models/tabnet/pytorch-training-2025-03-30-06-45-49-350/output/model.tar.gz"

# Create a PyTorchModel object
pytorch_model = PyTorchModel(
    model_data=model_path,
    role=role,
    entry_point="inference.py",  # Your inference script
    source_dir="./scripts",      # Folder containing inference.py
    framework_version="1.13.1",
    py_version="py39",
    sagemaker_session=sagemaker_session
)

# Deploy the model to a cheaper endpoint
predictor = pytorch_model.deploy(
    initial_instance_count=1,
    instance_type="ml.g4dn.4xlarge",  # Updated to cheaper instance ml.m5.large
    endpoint_name="tabnet-inference-fireguard-2025"
)

print(f"Model deployed at endpoint: {predictor.endpoint_name}")


Redeploying endpoint

In [2]:
import boto3

# Initialize SageMaker client
sagemaker_client = boto3.client('sagemaker')

# Deploy the endpoint using the existing endpoint configuration
endpoint_name = "tabnet-inference-fireguard-2025"
sagemaker_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_name  # Reusing the same endpoint configuration
)

print(f"Endpoint '{endpoint_name}' has been successfully redeployed.")

[04/06/25 20:55:16] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=19196;file:///home/vanel/anaconda3/envs/myenv/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=217740;file:///home/vanel/anaconda3/envs/myenv/lib/python3.10/site-packages/botocore/credentials.py#1278\1278]8;;\

Endpoint 'tabnet-inference-fireguard-2025' has been successfully redeployed.


## Testing endpoint script

In [ ]:
~ $ aws sagemaker list-endpoint-configs
{
    "EndpointConfigs": [
        {
            "EndpointConfigName": "tabnet-inference-fireguard-2025",
            "EndpointConfigArn": "aws:sagemaker:placeholder",
            "CreationTime": "2025-04-05T23:56:36.750000+00:00"
        }
    ]
}
~ $ Click inside the terminal window to reconnect and continue using your CloudShell session.


In [18]:
import gc
gc.collect()


266

### Getting data test endpoint

In [27]:
import pandas as pd
import s3fs

# Specify the bucket and file path
bucket = 'fireguarddata'
file_key = 'data/preprocessed_data/test.csv'
s3_path = f's3://{bucket}/{file_key}'

# Load the first 10 rows
df = pd.read_csv(s3_path, storage_options={"anon": False}, nrows=50)

[04/05/25 21:01:06] INFO     Skipping checksum validation. Response did not contain one of the  ]8;id=256920;file:///home/vanel/anaconda3/envs/myenv/lib/python3.10/site-packages/aiobotocore/httpchecksum.py\httpchecksum.py]8;;\:]8;id=530140;file:///home/vanel/anaconda3/envs/myenv/lib/python3.10/site-packages/aiobotocore/httpchecksum.py#107\107]8;;\
                             following algorithms: ['crc32', 'sha1', 'sha256'].                                    

In [28]:
import pandas as pd
pd.set_option('display.max_columns', None)
df.head()

,latitude,longitude,u10,v10,sp,lai_hv,lai_lv,tvl,cl,swvl1,daynight,tvh,ie,d2m,t2m,tcc,tcrw,rsn,sd,tsn,slt,year,month,day,skt,wind_speed,wind_direction,fuel_load,fuel_availability,fire_occurrence
0,0.294445,-0.181315,1.105994,0.030250,-0.391024,0.238015,-1.540314,-1.726427,-0.348803,0.017622,0.197106,-0.460135,0.573208,-0.104119,-0.300812,1.313689,-0.102781,-0.275318,-0.183884,0.086247,-0.852486,1.404559,0.660283,0.100102,-0.310211,0.233694,0.110108,-0.508083,-0.436723,0
1,1.228873,-0.181315,0.422261,-0.370437,-1.293002,1.351798,-0.707372,0.358779,0.115079,-0.825049,0.197106,-0.460135,0.560144,-0.379434,0.211126,0.733263,-0.102781,-0.275318,-0.183884,0.081971,0.226040,-0.834424,0.348730,1.357271,-0.228141,-0.544130,-0.184416,0.793895,1.240046,0
2,-0.826868,0.209589,-0.142371,-0.552047,1.245339,0.517575,0.009027,0.169215,-0.402946,-0.256516,0.197106,1.764457,-0.254320,-0.130492,-0.318492,-0.109391,-0.102781,-0.275318,-0.183884,-0.147731,-0.852486,0.444995,-1.209036,0.557254,0.314429,-0.867288,-0.687428,0.432034,0.595501,0
3,-0.639983,0.502766,-0.815843,0.014277,-0.395882,-0.006651,0.573114,-0.399478,-0.409454,-0.336696,0.197106,-0.460135,0.578964,-0.039654,-0.264546,-0.813935,-0.102781,-0.275318,-0.183884,0.039644,-0.852486,-0.194714,0.971836,0.785831,-0.421804,-0.919965,1.932404,0.256764,0.449127,0
4,0.387888,-0.572219,0.218778,1.040761,1.421688,-0.320773,1.094132,0.169215,0.225518,-0.391728,0.197106,1.764457,0.537422,0.118664,-0.696546,0.006109,-0.102781,-0.275318,-0.183884,0.086568,-0.852486,-1.474133,-0.897483,-1.157067,-0.778860,-0.026660,0.763427,0.235486,0.448969,0


In [29]:
# Drop the target column ('fire_occurrence') and convert to list of lists
input_data = df.drop(columns=["fire_occurrence"]).values.tolist()

# Format for the endpoint
sample_input = {
    "inputs": input_data
}

print(sample_input)


{'inputs': [[0.2944452, -0.18131496, 1.1059935, 0.030249966, -0.39102447, 0.2380152, -1.540314, -1.7264272, -0.34880304, 0.017622191, 0.19710617, -0.46013513, 0.57320803, -0.10411908, -0.30081216, 1.3136891, -0.10278146, -0.27531797, -0.18388379, 0.086246915, -0.8524859, 1.4045594, 0.66028297, 0.10010186, -0.31021142, 0.2336944, 0.11010842, -0.5080828, -0.4367233], [1.2288733, -0.18131496, 0.42226106, -0.3704374, -1.2930021, 1.3517978, -0.7073719, 0.35877916, 0.1150793, -0.82504874, 0.19710617, -0.46013513, 0.56014436, -0.37943405, 0.21112569, 0.7332627, -0.10278146, -0.27531797, -0.18388379, 0.081971385, 0.22604011, -0.8344239, 0.34872982, 1.3572712, -0.22814143, -0.5441299, -0.18441598, 0.7938955, 1.2400457], [-0.8268684, 0.20958863, -0.14237095, -0.55204684, 1.2453392, 0.5175745, 0.009026874, 0.16921492, -0.40294617, -0.2565162, 0.19710617, 1.7644571, -0.25432026, -0.1304918, -0.31849208, -0.10939122, -0.10278146, -0.27531797, -0.18388379, -0.14773142, -0.8524859, 0.44499516, -1.209

### Invoking the endpoint script

In [ ]:
import boto3
import json

# Endpoint name (from your deployment)
endpoint_name = "tabnet-inference-fireguard-2025"

# Create a SageMaker runtime client
runtime = boto3.client("sagemaker-runtime", region_name="us-east-1")

# Invoke the endpoint
response = runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/json",
    Body=json.dumps(sample_input)
)

# Parse the response
result = json.loads(response['Body'].read().decode())
print("Predictions:", result["predictions"])


### What is an Inference Recommender job?
The SageMaker Inference Recommender is a feature that helps you choose the optimal instance type and configuration for deploying your model. It does this by:
1. Testing your model on different instances.
2. Measuring performance metrics like latency, throughput, and cost.
3. Recommending the best instance based on your workload and model requirements.

#### Steps to Create and Upload the Sample Payload
- take 10 rows from your test.csv file as a sample.
- Save it as sample_input.csv.
- Compress the CSV File - Use tar to create a gzip archive:


In [31]:
import pandas as pd

# Load your test data from S3
bucket = "fireguarddata"
file_key = "data/preprocessed_data/test.csv"
s3_path = f"s3://{bucket}/{file_key}"

# Load the first 10 rows
df = pd.read_csv(s3_path, nrows=10)

# Drop the 'fire_occurrence' column
df = df.drop(columns=["fire_occurrence"], errors="ignore")

# Save as the sample input file
df.to_csv("sample_input.csv", index=False)
print("Sample input saved as sample_input.csv without the target column")


[04/05/25 21:14:43] INFO     Found credentials in shared credentials file: ~/.aws/credentials    ]8;id=666340;file:///home/vanel/anaconda3/envs/myenv/lib/python3.10/site-packages/aiobotocore/credentials.py\credentials.py]8;;\:]8;id=7632;file:///home/vanel/anaconda3/envs/myenv/lib/python3.10/site-packages/aiobotocore/credentials.py#621\621]8;;\

[04/05/25 21:14:46] INFO     Skipping checksum validation. Response did not contain one of the  ]8;id=987205;file:///home/vanel/anaconda3/envs/myenv/lib/python3.10/site-packages/aiobotocore/httpchecksum.py\httpchecksum.py]8;;\:]8;id=18962;file:///home/vanel/anaconda3/envs/myenv/lib/python3.10/site-packages/aiobotocore/httpchecksum.py#107\107]8;;\
                             following algorithms: ['crc32', 'sha1', 'sha256'].                                    

Sample input saved as sample_input.csv without the target column


In [32]:
#compress the CSV file
!tar -czvf sample_payload.tar.gz sample_input.csv

sample_input.csv


In [33]:
#upload to s3
import boto3

s3 = boto3.client("s3")
bucket_name = "fireguarddata"
s3_path = "sample_input/sample_payload.tar.gz"

s3.upload_file("sample_payload.tar.gz", bucket_name, s3_path)
print("Sample payload uploaded to S3!")

#final path - s3://fireguarddata/sample_input/sample_payload.tar.gz


Sample payload uploaded to S3!


then enter in  recommended job; After filling these, click "Run job" to start the inference recommender
- s3://fireguarddata/sample_input/sample_payload.tar.gz
- text/csv 


### Shutting down endpoint

In [ ]:
import boto3

# Specify your endpoint name
endpoint_name = "tabnet-inference-fireguard-2025"

# Initialize the SageMaker client
sagemaker_client = boto3.client("sagemaker")

# Update endpoint to zero instances
sagemaker_client.update_endpoint_weights_and_capacities(
    EndpointName=endpoint_name,
    DesiredWeightsAndCapacities=[
        {
            "VariantName": "AllTraffic",  # Default variant name
            "DesiredInstanceCount": 0,    # Scale down to zero
        }
    ]
)

print(f"Endpoint '{endpoint_name}' has been scaled down to zero instances.")


restart it

In [ ]:
sagemaker_client.update_endpoint_weights_and_capacities(
    EndpointName=endpoint_name,
    DesiredWeightsAndCapacities=[
        {
            "VariantName": "AllTraffic",
            "DesiredInstanceCount": 1,  # Scale back up to one
        }
    ]
)

print(f"Endpoint '{endpoint_name}' has been restarted with one instance.")


delete endpoint

In [ ]:
# Delete the endpoint
sagemaker_client.delete_endpoint(EndpointName=endpoint_name)
print(f"Endpoint '{endpoint_name}' deleted.")
